In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statistics
import seaborn as sns

# Import specific libraries for map plots
import folium
import plotly.express as px

# Connect to our Google drive account
from google.colab import drive
drive.mount('/content/drive')

# Import the original datasets as pandas dataframes
airports_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GSOM_Pirelli_2023/airports.csv', delimiter=",",header=0) # Header=0 means that the first row will be used as header for the dataset
flights_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GSOM_Pirelli_2023/Flights_Jan_cropped.csv', delimiter=",", header=0)

# **Check what I've imported**

In [ ]:
# AIRPORTS dataset - Verify that the dataset has been imported properly
airports_df.head()

In [ ]:
airports_df.describe()

In [ ]:
# Flights dataset - Verify that the dataset has been imported properly
flights_df.head()

In [ ]:
flights_df.describe()

In [ ]:
# NaN removal (not mandatory)
flights_df = flights_df.dropna().reset_index(drop=True)
flights_df.describe()


## **1.1 Plot all the airports on a geographic map**


In [ ]:
# Documentation Folium library: https://python-visualization.github.io/folium/

# Creation of the map
map = folium.Map(zoom_start=5, control_scale = 'True',max_bounds=True)

# Create the markers on the map
for index, row in airports_df.iterrows():
    visualization_string = row['AIRPORT'] + '('+row['DISPLAY_AIRPORT_CITY_NAME_FULL']+')'
    folium.Marker(location=[row['LATITUDE'], row['LONGITUDE']], popup= visualization_string , icon=folium.Icon(color='blue', icon='plane')).add_to(map)

# Show the map
map


# **1.2 Plot and analyze the distance of the flights**


In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
plt.hist(flights_df['DISTANCE'], bins=100, color='lightgrey', edgecolor='grey', linewidth=0.2)
ax.set_xlabel('Distance (mi)', fontsize=20)
ax.set_ylabel('Number of flights', fontsize=20)
ax.set_title('Distribution of Flight distances', fontsize=30)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

#statistics of the flight distance
avg_distance = flights_df['DISTANCE'].mean()
max_distance = flights_df['DISTANCE'].max()
min_distance = flights_df['DISTANCE'].min()

#Find the airports included in the longest & shortest flights
longest_flight = flights_df.loc[flights_df['DISTANCE'] == max_distance]
shortest_flight = flights_df.loc[flights_df['DISTANCE'] == min_distance]
longest_origin = longest_flight['ORIGIN'].iloc[0]
shortest_origin = shortest_flight['ORIGIN'].iloc[0]
longest_destination = longest_flight['DESTINATION'].iloc[0]
shortest_destination = shortest_flight['DESTINATION'].iloc[0]

# Plot vertical lines to show the mean, max, and min
ax.axvline(avg_distance, color='blue', linestyle='dashed', linewidth=2, label=f'Avgerage distance: {avg_distance:.2f} miles')
ax.axvline(max_distance, color='red', linestyle='dotted', linewidth=2, label=f'Maximum distance:{max_distance:.2f} miles\nBetween {longest_origin} and {longest_destination}')
ax.axvline(min_distance, color='green', linestyle='dashdot', linewidth=2, label=f'Minimum distance:{min_distance:.2f} miles\nBetween {shortest_origin} and {shortest_destination}')
plt.legend(fontsize=20)
plt.show()

# **2. Analyze number of flights**
Which is the airport with highest frequency of departures? \\
Which is the airport with highest frequency of arrivals? \\
Which company has more flights?

In [ ]:
departures_count = flights_df.groupby(['ORIGIN']).size().reset_index(name='Departures')
print(departures_count)
departures_count_arr = departures_count.to_numpy()


In [ ]:
#plot number of dep. flights for each airport
plt.figure(figsize=(20, 10))
plt.bar(departures_count_arr[:,0],departures_count_arr[:,1])
plt.ylabel('Number of flights')
plt.show()

In [ ]:
#plot number of dep. flights for each airport
plt.figure(figsize=(20, 10))
sorted_dep_count = departures_count.sort_values(by=['Departures'],ascending=False)
sorted_dep_count_arr = sorted_dep_count.to_numpy()

plt.figure(figsize=(20, 10))
plt.bar(sorted_dep_count_arr[:10,0],sorted_dep_count_arr[:10,1])
plt.ylabel('Number of flights',fontsize=20)
plt.show()

In [ ]:
#Create a dataset grouping by the "Destination" to count the number of flights reaching that airport
arrivals_count = flights_df.groupby(['DESTINATION']).size().reset_index(name='Arrivals')
print(arrivals_count)

In [ ]:
#plot number of arrival flights for each airport
plt.figure(figsize=(20, 10))
sorted_arrival_count = arrivals_count.sort_values(by=['Arrivals'],ascending=False)
sorted_arrival_count_arr = sorted_arrival_count.to_numpy()

plt.figure(figsize=(20, 10))
plt.bar(sorted_arrival_count_arr[:10,0],sorted_arrival_count_arr[:10,1])
plt.ylabel('Number of flights',fontsize=20)
plt.show()

In [ ]:
#Create a dataset grouping by the "Airline" to count of the number of flights for a single company
flights_count_per_operator = flights_df.groupby(['AIRLINE']).size().reset_index(name='Flights')
print(flights_count_per_operator)

Which are the most common routes?

In [ ]:
#Group by origin and destination airports and count the number of flights
grouped_df = flights_df.groupby(['ORIGIN', 'DESTINATION']).size().reset_index(name='num_flights_per_route')

sorted_df = grouped_df.sort_values('num_flights_per_route', ascending=False)

#Print the top 20 routes
print(sorted_df.head(20))

# **3. Analysis on flight delay**

average duration of a flight

In [ ]:
# Actual duration
mean_actual_duration = flights_df['ACTUAL_ELAPSED_TIME'].mean()
print("Mean of actual duration:", int(mean_actual_duration), 'minutes')

# Planned duration
mean_planned_duration = flights_df['SCHEDULED_ELAPSED_TIME'].mean()
print("Mean of planned duration:", int(mean_planned_duration), 'minutes')

In [ ]:
x1 = flights_df['ACTUAL_ELAPSED_TIME']
x2 = flights_df['SCHEDULED_ELAPSED_TIME']
count, bins_count = np.histogram(x1[~np.isnan(x1)], bins=100)
pdf = count / sum(count)
cdf = np.cumsum(pdf)
plt.plot(bins_count[1:], cdf, label="Actual")
count, bins_count = np.histogram(x2[~np.isnan(x2)], bins=100)
pdf = count / sum(count)
cdf = np.cumsum(pdf)
plt.plot(bins_count[1:], cdf, label="Scheduled")
plt.xlabel('Time [min]', fontsize=16)
plt.ylabel('%', fontsize=16)
plt.legend()
plt.show()

How much longer than expected each flight took? on average, and the maximum one

In [ ]:
diff = np.array(flights_df['ACTUAL_ELAPSED_TIME'] - flights_df['SCHEDULED_ELAPSED_TIME'])

plt.figure(figsize=(20, 10))
plt.hist(diff, bins=75, color='grey', edgecolor='lightgrey', density= True)
plt.xlabel('Difference (minutes)', fontsize=1)
plt.ylabel('Frequency', fontsize = 16)
plt.title('Expected vs actual flight time', fontsize=30)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
avg_diff = round(diff.mean(),2)
max_diff = round(diff.max(),2)

print('Mean difference between actual and planned durations is', avg_diff, 'minutes.')
print('Maximum difference between actual and planned durations is', max_diff, 'minutes.')


plt.figure(figsize=(20, 10))
plt.hist(diff, bins=75, color='grey', edgecolor='lightgrey', density= True)
plt.xlabel('Difference (minutes)', fontsize=1)
plt.ylabel('Frequency', fontsize = 16)
plt.title('Expected vs actual flight time', fontsize=30)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.axvline(avg_diff, color='green', linestyle='dashed', linewidth=2, label=f'Mean: {format(avg_distance, ".2f")} minutes')
plt.axvline(max_diff, color='red', linestyle='dashed', linewidth=2, label=f'Maximum: {format(max_distance, ".2f")} minutes')
plt.legend(fontsize = 20)
plt.show()

What is the percentage of flights that arrived earlier?

In [ ]:
num_ahead = (flights_df['ARRIVAL_DELAY'] < 0).sum()
pct_ahead = num_ahead / len(flights_df) * 100
print(num_ahead, "flights out of ",len(flights_df)," are ahead of time, i.e., " "{:.2f}%.".format(pct_ahead))

Is there a relationship between departure and arrival delay?

In [ ]:
correlationMatrix = flights_df.corr()

mask = np.triu(np.ones_like(correlationMatrix, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
colorPalette = sns.diverging_palette(240, 10)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(correlationMatrix, annot=True, cmap=colorPalette, vmax=1, vmin=-1, center=0, linewidths=.15)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(flights_df['DEPARTURE_DELAY'], flights_df['ARRIVAL_DELAY'], color = 'steelblue')
plt.xlabel('Departure delay [min]', fontsize = 15)
plt.ylabel('Arrival delay [min]', fontsize = 15)
plt.show()

Find the airport with highest departure delay and derive statistics on such delay.

In [ ]:
max_delay = flights_df['DEPARTURE_DELAY'].max()
flight_with_highest_dep_delay = flights_df[flights_df['DEPARTURE_DELAY']==max_delay]
print(flight_with_highest_dep_delay)

# **Analysis on flight cancellation**


Which is the percentage of cancelled flights in USA?

In [ ]:
cancelled_flight = flights_df[flights_df['CANCELLED']==1]
print("The amount of cancelled flight is", cancelled_flight['CANCELLED'].count(),"out of ",len(flights_df)," " )

perc = cancelled_flight['CANCELLED'].count()*100/len(flights_df)
print("The percentage of cancelled flights is", round(perc,2) ,"%")


In [25]:
#re-upload the dataset
flights_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GSOM_Pirelli_2023/Flights_Jan.csv', delimiter=",", header=0)

In [ ]:
cancelled_flight = flights_df[flights_df['CANCELLED']==1]
print("The amount of cancelled flight is", cancelled_flight['CANCELLED'].count(),"out of ",len(flights_df)," " )

perc = cancelled_flight['CANCELLED'].count()*100/len(flights_df)
print("The percentage of cancelled flights is", round(perc,2) ,"%")


Which airport is more subject to flight cancellation?

In [ ]:
dataset_of_canc_flights = flights_df[flights_df["CANCELLED"]==1]

cancel_flight_per_airport = dataset_of_canc_flights.groupby("ORIGIN")["ORIGIN"].count().reset_index(name='Canc_Flights')

print(cancel_flight_per_airport)

In [ ]:

sorted_cancel_flight_per_airport = cancel_flight_per_airport.sort_values(by=['Canc_Flights'],ascending=False)
sorted_cancel_flight_per_airport = sorted_cancel_flight_per_airport.to_numpy()

plt.figure(figsize=(20, 10))
plt.bar(sorted_cancel_flight_per_airport[:10,0],sorted_cancel_flight_per_airport[:10,1])
plt.ylabel('Number of cancelled flights',fontsize=20)
plt.show()


Is there a day with remarkably high  number of cancelled flights?

In [ ]:
flights_df['SCHEDULED_DEPARTURE_TIME'] = pd.to_datetime(flights_df['SCHEDULED_DEPARTURE_TIME'])

flights_df['SCHEDULED_DEPARTURE_TIME'] = pd.to_datetime(flights_df['SCHEDULED_DEPARTURE_TIME'], format='%Y-%m-%d %H:%M:%S')

cancelled_by_day = flights_df.groupby(flights_df['SCHEDULED_DEPARTURE_TIME'].dt.date)['CANCELLED'].sum()
x_labels = [date.strftime('%b %d') for date in cancelled_by_day.index]

max_cancelled_day = cancelled_by_day.idxmax()
max_cancelled_flights = cancelled_by_day[max_cancelled_day]
print("The day with the highest number of cancelled flights is:", max_cancelled_day, "with", max_cancelled_flights, "cancelled flights.")


# **Decision task**

Possible analyses:

1.   Analysis of the flows
2.   Analysis of delays
3.   Analysis of the cost of land
4.   Analysis of the weather
5.   Analysis of airport proximity

A high flight number, both in terms of arrivals and departures, makes a zone more appealing for a new airport as the location is likely to be in a favorable geographical position in terms of weather, as studies on existing airports in the region have already been conducted, or the zone and nearby cities are likely to be appealing for visitors (either business or vacation), or it may be in a strategic position to allow stopovers.



In [30]:
flights_df = flights_df.dropna().reset_index(drop=True)

In [ ]:
# Count the number of arrivals and departures for each airport
count_arr = flights_df["DESTINATION"].value_counts()
count_dep = flights_df["ORIGIN"].value_counts()

df = pd.concat([count_arr, count_dep], axis=1)
df["TOTAL"]=df["DESTINATION"]+df["ORIGIN"]
df = df.reset_index()
df = df.rename(columns={'index': 'AIRPORT'})
merged_dataset = pd.merge(df, airports_df, on='AIRPORT')
merged_dataset = merged_dataset[["AIRPORT", "TOTAL", "LATITUDE", "LONGITUDE"]]
merged_dataset = merged_dataset.sort_values(by='TOTAL', ascending=False)

#density_df=merged_dataset

map_airports = folium.Map(location=[40.693943, -73.985880], zoom_start=4)
for index, row in merged_dataset.iterrows():
    folium.CircleMarker(location=[row['LATITUDE'], row['LONGITUDE']], radius=row['TOTAL']/2000,
                        color='#FF5733', fill=True, fill_color='#FF5733').add_to(map_airports)
display(map_airports)


In [ ]:
top10_density=merged_dataset["AIRPORT"].head(10)
worst10_density=merged_dataset["AIRPORT"].tail(10)
print('Top 10 Airports by flights density')
print(top10_density.to_string(index=False))
top10_density=merged_dataset["AIRPORT"].head(10)
print('\nWorst 10 Airports by flights density:')
print(worst10_density.to_string(index=False))

A second parameter taken into account is the sum of the total flights delay, intended as the sum of arrival delay (which directly impact on the customers) and departure delay (which still is an indicator of bad performances and overcrowding for the airport).

Flight delay analysis is a measure of the efficiency and quality of an airport. It is essential to measure flight delays as they can have a significant impact on both airlines and passengers, leading to inconvenience, missed connections, and additional costs. Higher delays imply poorer performance and quality of an airport.



In [33]:
merged_df = pd.merge(flights_df, airports_df, how='inner', left_on='ORIGIN', right_on='AIRPORT')
merged_df = pd.merge(merged_df, airports_df, how='inner', left_on='DESTINATION', right_on='AIRPORT', suffixes=('_origin', '_destination'))

grouped_df = merged_df.groupby(['AIRPORT_origin', 'LATITUDE_origin', 'LONGITUDE_origin']).agg({'DEPARTURE_DELAY': 'sum', 'ARRIVAL_DELAY': 'sum'}).reset_index()
grouped_df.rename(columns={'AIRPORT_origin': 'AIRPORT', 'LATITUDE_origin': 'LATITUDE', 'LONGITUDE_origin': 'LONGITUDE'}, inplace=True)

max_delay = grouped_df['ARRIVAL_DELAY'].max() + grouped_df['DEPARTURE_DELAY'].max()
grouped_df['DELAY_SEVERITY'] = (grouped_df['ARRIVAL_DELAY'] + grouped_df['DEPARTURE_DELAY']) / max_delay

#delay_df=grouped_df

map_airports = folium.Map(location=[40.693943, -73.985880], zoom_start=4)

for index, row in grouped_df.iterrows():
        folium.CircleMarker(location=[row['LATITUDE'], row['LONGITUDE']], radius=row['DELAY_SEVERITY']*75,
                            color='#FF5733', fill=True, fill_color='#FF5733').add_to(map_airports)
display(map_airports)

sorted_df = grouped_df.sort_values(by=['ARRIVAL_DELAY', 'DEPARTURE_DELAY'], ascending=False)
top_10_delay = sorted_df.head(10).reset_index()



In [ ]:
print('Top 10 Airports with highest delay:')
print(top_10_delay['AIRPORT'].to_string(index=False))


A third parameter to be considered can the cost of land per acre in the area where the airport will be built.
The level of granularity we considered for that decision is the "State" level.

The map shows the most expensive in red and the most cheap in green. Additionally, we maintain a dataset that includes the five most expensive areas and the five most cheap for reference.

*Data source: https://www.landsearch.com/price*

In [ ]:
state_names = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts',
    'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
    'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
    'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

land_costs = {'AL': 16039, 'AK': 10295, 'AZ': 20070, 'AR': 14174, 'CA': 23687, 'CO': 9792,
              'CT': 30694, 'DE': 78258, 'FL': 64690, 'GA': 22607, 'HI': 80355, 'ID': 19940,
              'IL': 36882, 'IN': 28759, 'IA': 22053, 'KS': 7352, 'KY': 10688, 'LA': 14211,
              'ME': 8903, 'MD': 34637, 'MA': 45591, 'MI': 14203, 'MN': 18281, 'MS': 9014,
              'MO': 12624, 'MT': 7756, 'NE': 10385, 'NV': 15923, 'NH': 18315, 'NJ': 92181,
              'NM': 4988, 'NY': 14772, 'NC': 25626, 'ND': 3993, 'OH': 23129, 'OK': 6730,
              'OR': 11572, 'PA': 20772, 'RI': 75341, 'SC': 27583, 'SD': 14134, 'TN': 12074,
              'TX': 17935, 'UT': 30149, 'VT': 10696, 'VA': 16015, 'WA': 19604, 'WV': 11561,
              'WI': 17891, 'WY': 14427}

# create a pandas dataframe from the land_costs dictionary
df = pd.DataFrame.from_dict(land_costs, orient='index', columns=['LandCosts'])

# add a column for the state abbreviations, which will be used as the location identifier
df.reset_index(inplace=True)
df.rename(columns={'index': 'State'}, inplace=True)
df_sorted = df.sort_values(by='LandCosts', ascending=True)

# create the choropleth map
fig = px.choropleth(df,
                    locations='State',
                    locationmode='USA-states',
                    color='LandCosts',
                    scope='usa',
                    hover_name='State',
                    color_continuous_scale='Viridis',
                    range_color=[df['LandCosts'].min(), df['LandCosts'].max()],
                    labels={'LandCosts': 'Land Costs ($)'})


fig.update_layout(title_text='Average land cost per acre at state-level',
                  title_font_size=30,
                  title_font_color='black',
                  title_x=0.45,
                  width=800,
                  height=500)
fig.show()

top_5_highest_cost = df_sorted.nlargest(5, 'LandCosts')
top_5_lowest_cost = df_sorted.nsmallest(5, 'LandCosts')
print('Top 5 States with the highest cost:')
print(top_5_highest_cost['State'].map(state_names).to_string(index=False))
print('\nTop 5 States with the lowest cost:')
print(top_5_lowest_cost['State'].map(state_names).to_string(index=False))

A fourth parameter can be the weather condition, which encompasses factors such as precipitation, temperature, and wind speed. Focusing on precipitation (as potentially higher affecting the flights) we have the following map

*Data source: https://www.currentresults.com/Weather/US/average-annual-state-temperatures.php*

In [ ]:
import json
with open('/content/drive/MyDrive/Colab Notebooks/GSOM_Pirelli_2023/us-states.json') as f:
   counties = json.load(f)


df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GSOM_Pirelli_2023/NaturalHazard.csv", delimiter=',',dtype={"State": str})
df = df[['State','Station.ID', 'Milli­metres']]
df['Station.ID'] = df['Station.ID'].str.strip()
df = df.sort_values(by='Milli­metres', ascending=True)


fig = px.choropleth(df,
                    locations ="Station.ID",
                    locationmode='USA-states',
                    color='Milli­metres',
                    scope= "usa",
                    hover_name='State',
                    color_continuous_scale="Viridis",
                    labels={'Millimiters'},
                    range_color=[df['Milli­metres'].min(), df['Milli­metres'].max()]
                    )

fig.update_layout(
      title_text = 'Average millimeters of rain per year',
      title_font_size = 30,
      title_font_color="black",
      title_x=0.45,
      width=800,
      height=500
         )
fig.show()



In [ ]:
top10_rain = df.nlargest(10, 'Milli­metres')
print('Top 10 States with the highest millimetres of rain per year:')
print(top10_rain["State"].to_string(index=False))

A fifth parameter in selecting a potential new airport location can be the proximity to other existing airports. This analysis is more granular, taking into account the potential issues that may arise from having two airports too close together, such as overlapping flight departures, congestion, and delays. Therefore, it is recommended to place the new airport at least on the other side of the city to avoid these issues and ensure smooth operations. The following graphs shows the airports locations and the centroid of the geographical coordinates per each state.

In [38]:
import geopy
import geopy.distance
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

airports_df[["NAME","STATE"]] = airports_df.DISPLAY_AIRPORT_CITY_NAME_FULL.str.split(",",expand=True)

list = airports_df[['AIRPORT','LATITUDE','LONGITUDE','STATE']].sort_values(['STATE','AIRPORT'], ascending=True).to_numpy().tolist()

source = []
target = []
km = []
miles = []
state = []

for i in list:
  for j in list[1:]:
    if i[3]==j[3] and i[0] != j[0]:
        source.append(i[0])
        target.append(j[0])
        km.append(float(geopy.distance.geodesic((i[1],i[2]), (j[1],j[2])).km))
        miles.append(float(geopy.distance.geodesic((i[1],i[2]), (j[1],j[2])).miles))
        state.append(i[3])

state_x = np.array(state)
state_x = np.unique(state_x)

df = pd.DataFrame({'SOURCE': source,
                   'TARGET': target,
                   'KM': km,
                   'MILES': miles,
                   'STATE': state})

In [ ]:
!pip install basemap
from mpl_toolkits.basemap import Basemap

fig = plt.figure(figsize=(20,10))

map = Basemap(llcrnrlon=-119,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=49,projection='lcc',lat_1=32,lat_2=45,lon_0=-95)
map.shadedrelief()
map.drawcoastlines()
map.drawcountries(linewidth = 3)
map.drawstates(color='0.3')

for index, (code, y,x) in airports_df[['AIRPORT', 'LATITUDE', 'LONGITUDE']].iterrows():
    x, y = map(x, y)
    map.plot(x, y, marker='o', markersize = 6, markeredgewidth = 1, color = 'red')

def centeroidnp(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    return sum_x/length, sum_y/length

for i in state_x:

  df_state = airports_df[airports_df['STATE'].str.strip()==str(i).strip()]
  point_list = df_state[['LATITUDE','LONGITUDE']].to_numpy()
  latitude, longitude = centeroidnp(point_list)
  x1, y1 = map(longitude,latitude)
  map.plot(x1, y1, marker='D', markersize = 15, color = 'blue', markeredgecolor='k')

plt.show()

Analyzing the traffic and delays, a promising area to place a new airport seems to be Chicago. The two main airports (MDW and ORD) of the city seem to have difficulties in handling the flows of flights and have a high demand.

 The analysis of flows, cancellations and delays revealed that
*   Illinois is the 4th state with the highest flights demand
*   The maximum number of cancelled flights with cancellation code 'A' or 'C' is observed in ORD airport
*   The percentages of delayed flights in ORD and MDW are close to the maximum value of the dataset
*   The maximum number of highly delayed flights is observed in ORD airport


Illinois is the 4th state with the highest flights demand. As the demand is high it might be assumed that building a new airport there would be economically sustainable. Indeed, many flights could potentially be performed from/to that new airport.

Expected capacity of the new airport in Chicago.

The elements considered in this analysis are:
* The size of the two existing airports in Chicago and their expected growth
* Their cancelled flights with cancellation code 'A' or 'C'
* Their flights delayed for more than 45 minutes

> To determine the minimum capacity of the new airport,  first consider the sum of the cancelled flights with cancellation code A and C, and the flights delayed by more than 45 minutes.
\
\
This sum represents the flights that could be transferred from the existing airports to the new one. The objective would be to improve the performances of the operations of the existing airports and to avoid further cancellations and/or delays.

The assumption is made that 45 minutes of delay represents the starting point of operational problems in executing and planning activities of flights.

In [40]:
flights_ORD = flights_df[flights_df['ORIGIN']=='ORD'].reset_index(drop=True)
flights_MDW = flights_df[flights_df['ORIGIN']=='MDW'].reset_index(drop=True)
chicago_flights = pd.concat([flights_ORD, flights_MDW], axis=0).reset_index(drop=True)

In [ ]:
print(flights_ORD)


In [ ]:
print(flights_MDW)

In [ ]:
print(chicago_flights)

In [ ]:
CHI_lat = 41.881832
CHI_long =  -87.623177

map = folium.Map(location=[CHI_lat, CHI_long], zoom_start=6, control_scale='True', max_zoom=20, min_zoom=2, max_bounds=True)

for index, row in airports_df.iterrows():
    str_air = row['AIRPORT'] + ' (' + row['DISPLAY_AIRPORT_NAME'] + ')'+' (' + row['DISPLAY_AIRPORT_CITY_NAME_FULL'] + ')'
    folium.Marker(location=[row['LATITUDE'], row['LONGITUDE']], popup= str_air).add_to(map)

# Add the circle in Chicago representing the area of influence of ORD and MDW airports
folium.Circle(location=[CHI_lat, CHI_long], radius=65000, color='red', fill=True, fill_opacity=0.2).add_to(map)

map

In [ ]:
delay_flights_CHI = chicago_flights[chicago_flights['DEPARTURE_DELAY']>45].reset_index(drop=True)
print(len(delay_flights_CHI), "flights out of ",len(chicago_flights)," in Chicago have a delay of more than 45 min")

In [ ]:
canc_flights_MDW = cancel_flight_per_airport[cancel_flight_per_airport['ORIGIN'] == 'MDW']
canc_flights_ORD = cancel_flight_per_airport[cancel_flight_per_airport['ORIGIN'] == 'ORD']
canc_flights_CHI = int(canc_flights_MDW['Canc_Flights']) + int(canc_flights_ORD['Canc_Flights'])

print(canc_flights_MDW)
print(canc_flights_ORD)
print(canc_flights_CHI, "flights out of ",len(chicago_flights)," in Chicago have been cancelled")

In [ ]:
print("The new airport in Chicago should accomodate for at least ",canc_flights_CHI+len(delay_flights_CHI),"flights per month, i.e., a capacity similar to MDW")